# VectorStores

Uma das maneiras mais comuns de armazenar e buscar dados não estruturados é realizando o embedding e armazenando os vetores resultantes e, em seguida, na hora da consulta, realizar o embedding da consulta e recuperar os vetores 'mais semelhantes'. Uma VectorStore faz o armazenamento dos vetores e a realização da busca de vetores para você

## Chroma VectorStore

In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Document Laoding

In [2]:
caminho = "arquivos/Explorando o Universo das IAs com Hugging Face.pdf"
loader = PyPDFLoader(caminho)
paginas = loader.load()

In [3]:
len(paginas)

89

### Text Splitting

In [4]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)
len(documents)

266

### Criando a VectorStore

In [ ]:
#!pip install pysqlite3-binary

In [5]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [7]:
from langchain_chroma import Chroma

diretorio = 'arquivos/chroma_vectorstore'

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [8]:
print(vectorstore._collection.count())

266


### Importando vectorstore do disco

In [9]:
diretorio = 'arquivos/chroma_vectorstore'

vectorstore = Chroma(
    embedding_function=embeddings_model,
    persist_directory=diretorio
)

### Retrieval

In [10]:
pergunta = 'O que é o Hugging Face?'

docs = vectorstore.similarity_search(pergunta, k=5)
len(docs)

5

In [11]:
for doc in docs:
    print(doc.page_content)
    print(f'====== {doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
01. O que é Hugging Face?
Bem-vindos ao curso de Hugging Face da Asimov Academy!
Neste curso, vamos explorar as principais utilidades da plataforma de IAHugging Face. Aprenderemos
como utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face
aos nossos scripts de Python.
Afinal, o que é Hugging Face?
A Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.
====== {'page': 5, 'source': 'arquivos/Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
Conteúdo
01. O que é Hugging Face? 5
Afinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
É tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
Como usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
02. A plataforma Hugging Face 7
====== {'page': 1, 'so

## FAISS VectorStore

https://python.langchain.com/docs/integrations/vectorstores/

In [12]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
caminho = "arquivos/Explorando o Universo das IAs com Hugging Face.pdf"
loader = PyPDFLoader(caminho)
paginas = loader.load()

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [15]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model
)

In [16]:
pergunta = 'O que é o Hugging Face?'

docs = vectorstore.similarity_search(pergunta, k=5)
len(docs)

5

In [17]:
for doc in docs:
    print(doc.page_content)
    print(f'====== {doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
01. O que é Hugging Face?
Bem-vindos ao curso de Hugging Face da Asimov Academy!
Neste curso, vamos explorar as principais utilidades da plataforma de IAHugging Face. Aprenderemos
como utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face
aos nossos scripts de Python.
Afinal, o que é Hugging Face?
A Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.
====== {'source': 'arquivos/Explorando o Universo das IAs com Hugging Face.pdf', 'page': 5}


Explorando o Universo das IAs com Hugging Face
Conteúdo
01. O que é Hugging Face? 5
Afinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
É tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
Como usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
02. A plataforma Hugging Face 7
====== {'source': 'arq

### Salvando bd FAISS

In [18]:
vectorstore.save_local('arquivos/faiss_bd')

### Importando bd FAISS

In [19]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.load_local(
    'arquivos/faiss_bd',
    embeddings=embeddings_model,
    allow_dangerous_deserialization=True
)

### Algumas limitações